# Injury Analysis:

- Identify the unique injuries present in the dataset for self assessment and the diagnostic of the professional medical reviewer.
- Provide a description of the injuries.
- Analyse the relationship between self-assessment and the diagnostic of the medical reviewer.
- Decide if self assessment adds information.

In [1]:
import pandas as pd
import os
from core.constants import RICKD_PROY_RESOURCE_FOLDER, RICKD_SOURCE_DATA_FOLDER, RICKD_PROCESSED_DATA_FOLDER, RICKD_SESSION_DATA_FULL_FILE

run_data_full = pd.read_csv(RICKD_SESSION_DATA_FULL_FILE)
run_injury_data = run_data_full[['id', 'sub_id','InjDefn', 'SpecInjury', 'SpecInjury2']]
run_injury_data

,id,sub_id,InjDefn,SpecInjury,SpecInjury2
0,100433_20101005t132240,100433,training volume/intensity affected,pain,NaN
1,100434_20101117t132240,100434,training volume/intensity affected,disc degeneration,NaN
2,100537_20120703t102550,100537,2 workouts missed in a row,other,NaN
3,100560_20120717t103748,100560,no injury,NaN,NaN
4,101481_20120717t105021,101481,no injury,NaN,NaN
...,...,...,...,...,...
1827,200986_20150312t143944,200986,no injury,NaN,NaN
1828,200987_20150312t160840,200987,continuing to train in pain,pelvic malalignment,itb syndrome
1829,201100_20150409t155915,201100,training volume/intensity affected,muscle strain,ligament sprain
1830,201101_20150413t143152,201101,no injury,NaN,NaN


## Unique Injury types:


In [2]:
# Combine SpecInjury and SpecInjury2 columns and get unique values
unique_combined_injuries = pd.concat([run_injury_data['SpecInjury'], run_injury_data['SpecInjury2']]).dropna().drop_duplicates()

print("### Distinct Injury Types")
for injury in sorted(unique_combined_injuries):
    print(f"- {injury}")


### Distinct Injury Types
- achilles tendonitis
- bursitis
- calf muscle strain
- chondromalacia
- compartment syndrome
- cyst
- deformity (e.g hammer/claw toes, bunion)
- deformity(ie hammer/claw toes, bunion)
- disc degeneration
- disc protrusion
- dislocation
- fibula fracture
- fibula stress fracture
- fill in specifics below
- first ray tendonitis (turf toe)
- gastrocnemius strain
- groin muscle strain
- hamstring muscle strain
- high ankle sprain with tendonosis
- hip flexor strain
- hip joint irritation
- ischial bursitis
- itb syndrome
- labral tear
- ligament sprain
- ligament tear/rupture acl
- ligament tear/rupture mcl
- ligament tear/rupture pcl
- ligament tear/rupture-acl
- low back pain
- medial tibial stress syndrome
- meniscal tear medial
- meniscal tear-lateral
- meniscal tear-medial
- metarsalgia
- metatarsal stress fracture
- muscle spasm
- muscle strain
- mva, not running related
- nerve impingement
- no injury
- non-specific injury
- osteitis pubis
- osteoarthritis

In [3]:

from core.data_quality import map_injury_codes
from core.constants import RICKD_MAP_INJURY_CODE, RICKD_MAP_INJURY_DESC, RICKD_MAP_SELF_INJURY_CODE


df = map_injury_codes(run_injury_data, RICKD_MAP_INJURY_CODE, "SpecInjury", "injury_code")
df = map_injury_codes(df, RICKD_MAP_INJURY_CODE, "SpecInjury2", "injury2_code")
df = map_injury_codes(df, RICKD_MAP_INJURY_DESC, "injury_code", "injury_desc")
df = map_injury_codes(df, RICKD_MAP_INJURY_DESC, "injury2_code", "injury2_desc")
df = map_injury_codes(df, RICKD_MAP_SELF_INJURY_CODE, "InjDefn", "self_injury_code")

df.to_csv(os.path.join(RICKD_PROCESSED_DATA_FOLDER, "tmp_injury_data.csv"))


In [4]:
print("\n### Distinct Injury Definitions")
print("```")
for injury_def in sorted(run_data_meta_cleaned['InjDefn'].dropna().unique()):
    print(f"- {injury_def}")
print("```")


### Distinct Injury Definitions
```


NameError: name 'run_data_meta_cleaned' is not defined